In [1]:
from utz import *
from njdot.crashes import Crashes, pk_cols, ksi_renames, road_cols
from njdot.data import cc2cn, cn2cc
from njdot.paths import CNS, CRASHES_DB
from nj_crashes.paths import PUBLIC_DIR

In [2]:
%%time
c = Crashes.load()
c

CPU times: user 25.8 s, sys: 15.5 s, total: 41.3 s
Wall time: 47.7 s


,year,cc,mc,case,cn,mn,pdc,pdn,station,tk,ti,pk,pi,severity,Intersection,Alcohol Involved,HazMat Involved,crash_type,tv,road,road_direction,Route,Route Suffix,sri,mp,Road System,Road Character,Road Surface Type,Surface Condition,Light Condition,Environmental Condition,Road Divided By,Temporary Traffic Control Zone,Distance To Cross Street,Unit Of Measurement,Direction From Cross Street,cross_street,Is Ramp,Ramp To/From Route Name,Ramp To/From Route Direction,Posted Speed,Posted Speed Cross Street,olat,olon,Cell Phone In Use Flag,Other Property Damage,Reporting Badge No.,dt,Road Horizontal Alignment,Road Grade,First Harmful Event
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,2001,1,1,#2001-17846,Atlantic,Absecon City,01,Absecon City,MUNICIPAL COMP?,0,0,0,0,p,B,False,False,03,2,CALDERON AVENUE,,,,,NaN,07,02,02,01,06,01,05,01,100,FE,N,RT 30,,,,25,,NaN,NaN,N,NEVER SAW V-1 MINOR DAMAGE - NO INJURIES REPOR...,830,2001-12-21 18:34:00,NaN,NaN,NaN
1,2001,1,1,01-00029,Atlantic,Absecon City,1,Absecon,,0,0,0,0,p,B,False,False,06,2,RITZ DRIVE,,,,,NaN,07,01,02,03,06,01,05,01,,,,,,,,25,,NaN,NaN,N,,836,2001-01-01 09:30:00,NaN,NaN,NaN
2,2001,1,1,01-004615,Atlantic,Absecon City,1,Absecon,,0,4,0,0,i,I,False,False,03,2,MORTON AVENUE,,,,,NaN,07,01,02,01,01,01,05,01,,AT,,NEW YORK AVENUE,,,,25,,NaN,NaN,N,,836,2001-04-10 14:44:00,NaN,NaN,NaN
3,2001,1,1,01-004880,Atlantic,Absecon City,1,Absecon,,0,1,0,0,i,B,False,False,01,2,RT 30,,30,,00000030__,51.10,02,01,02,01,01,01,05,01,,,,,,,,45,,NaN,NaN,N,WITNESS-PETRIA GIBSON AND ANGELO HERSH,886,2001-04-15 13:56:00,NaN,NaN,NaN
4,2001,1,1,01-004912,Atlantic,Absecon City,1,Absecon,,0,0,0,0,p,I,False,False,01,2,CR 651,,651,,,NaN,05,02,02,02,01,02,05,01,,AT,,CR 630,,,,35,,NaN,NaN,N,*BETH VEHICLES-RELIANCE INSURANCE CO.(215)864-...,836,2001-04-16 10:29:00,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6077298,2021,21,23,B150-2021-00117A,Warren,White Twp,02,New Jersey State Police,WASHINGTON - SQ,0,0,0,0,p,I,False,False,11,1,WARREN COUNTY 620,N,620,,21000620__,0.00,05,,02,01,06,01,05,01,,AT,,ROUTE 519 / S BRIDGEVILLE RD,,,,50,50,40.80707,-75.07559,N,Damage to 10ft of Country road guardrail.,7837,2021-05-26 21:08:00,02,04,46
6077299,2021,21,23,B150-2021-00118A,Warren,White Twp,02,New Jersey State Police,WASHINGTON - SQ,0,0,0,0,p,I,False,False,01,2,MANUNKA CHUNK RD / UPPER SEREPTA RD **,W,,,21231034__,1.73,07,,02,01,01,01,05,01,,AT,,MASSENATTS RD,,,,40,50,40.84466,-75.05549,N,,8610,2021-05-25 07:26:00,01,04,26
6077300,2021,21,23,B150-2021-00128A,Warren,White Twp,02,New Jersey State Police,WASHINGTON - SQ,0,0,0,0,p,B,False,False,12,1,ROUTE 519,S,519,,00000519__,44.00,05,,02,02,05,04,05,01,,,,,,,,50,,40.80564,-75.07627,N,,8514,2021-06-03 20:17:00,03,04,24


In [3]:
c[c.duplicated(keep=False, subset=pk_cols)]

,year,cc,mc,case,cn,mn,pdc,pdn,station,tk,ti,pk,pi,severity,Intersection,Alcohol Involved,HazMat Involved,crash_type,tv,road,road_direction,Route,Route Suffix,sri,mp,Road System,Road Character,Road Surface Type,Surface Condition,Light Condition,Environmental Condition,Road Divided By,Temporary Traffic Control Zone,Distance To Cross Street,Unit Of Measurement,Direction From Cross Street,cross_street,Is Ramp,Ramp To/From Route Name,Ramp To/From Route Direction,Posted Speed,Posted Speed Cross Street,olat,olon,Cell Phone In Use Flag,Other Property Damage,Reporting Badge No.,dt,Road Horizontal Alignment,Road Grade,First Harmful Event
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


In [4]:
type(c)

njdot.crashes.Crashes

In [5]:
assert c[['cc', 'cn']].set_index('cc')['cn'].groupby(lambda x:x).apply(lambda s: singleton(s.tolist())).to_dict() == cc2cn

In [6]:
gdf = c.gdf('o')
gdf

,year,cc,mc,case,cn,mn,pdc,pdn,station,tk,ti,pk,pi,severity,Intersection,Alcohol Involved,HazMat Involved,crash_type,tv,road,road_direction,Route,Route Suffix,sri,mp,Road System,Road Character,Road Surface Type,Surface Condition,Light Condition,Environmental Condition,Road Divided By,Temporary Traffic Control Zone,Distance To Cross Street,Unit Of Measurement,Direction From Cross Street,cross_street,Is Ramp,Ramp To/From Route Name,Ramp To/From Route Direction,Posted Speed,Posted Speed Cross Street,olat,olon,Cell Phone In Use Flag,Other Property Damage,Reporting Badge No.,dt,Road Horizontal Alignment,Road Grade,First Harmful Event,geometry
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,2001,1,1,#2001-17846,Atlantic,Absecon City,01,Absecon City,MUNICIPAL COMP?,0,0,0,0,p,B,False,False,03,2,CALDERON AVENUE,,,,,NaN,07,02,02,01,06,01,05,01,100,FE,N,RT 30,,,,25,,NaN,NaN,N,NEVER SAW V-1 MINOR DAMAGE - NO INJURIES REPOR...,830,2001-12-21 18:34:00,NaN,NaN,NaN,POINT EMPTY
1,2001,1,1,01-00029,Atlantic,Absecon City,1,Absecon,,0,0,0,0,p,B,False,False,06,2,RITZ DRIVE,,,,,NaN,07,01,02,03,06,01,05,01,,,,,,,,25,,NaN,NaN,N,,836,2001-01-01 09:30:00,NaN,NaN,NaN,POINT EMPTY
2,2001,1,1,01-004615,Atlantic,Absecon City,1,Absecon,,0,4,0,0,i,I,False,False,03,2,MORTON AVENUE,,,,,NaN,07,01,02,01,01,01,05,01,,AT,,NEW YORK AVENUE,,,,25,,NaN,NaN,N,,836,2001-04-10 14:44:00,NaN,NaN,NaN,POINT EMPTY
3,2001,1,1,01-004880,Atlantic,Absecon City,1,Absecon,,0,1,0,0,i,B,False,False,01,2,RT 30,,30,,00000030__,51.10,02,01,02,01,01,01,05,01,,,,,,,,45,,NaN,NaN,N,WITNESS-PETRIA GIBSON AND ANGELO HERSH,886,2001-04-15 13:56:00,NaN,NaN,NaN,POINT EMPTY
4,2001,1,1,01-004912,Atlantic,Absecon City,1,Absecon,,0,0,0,0,p,I,False,False,01,2,CR 651,,651,,,NaN,05,02,02,02,01,02,05,01,,AT,,CR 630,,,,35,,NaN,NaN,N,*BETH VEHICLES-RELIANCE INSURANCE CO.(215)864-...,836,2001-04-16 10:29:00,NaN,NaN,NaN,POINT EMPTY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6077298,2021,21,23,B150-2021-00117A,Warren,White Twp,02,New Jersey State Police,WASHINGTON - SQ,0,0,0,0,p,I,False,False,11,1,WARREN COUNTY 620,N,620,,21000620__,0.00,05,,02,01,06,01,05,01,,AT,,ROUTE 519 / S BRIDGEVILLE RD,,,,50,50,40.80707,-75.07559,N,Damage to 10ft of Country road guardrail.,7837,2021-05-26 21:08:00,02,04,46,POINT (-75.07559 40.80707)
6077299,2021,21,23,B150-2021-00118A,Warren,White Twp,02,New Jersey State Police,WASHINGTON - SQ,0,0,0,0,p,I,False,False,01,2,MANUNKA CHUNK RD / UPPER SEREPTA RD **,W,,,21231034__,1.73,07,,02,01,01,01,05,01,,AT,,MASSENATTS RD,,,,40,50,40.84466,-75.05549,N,,8610,2021-05-25 07:26:00,01,04,26,POINT (-75.05549 40.84466)
6077300,2021,21,23,B150-2021-00128A,Warren,White Twp,02,New Jersey State Police,WASHINGTON - SQ,0,0,0,0,p,B,False,False,12,1,ROUTE 519,S,519,,00000519__,44.00,05,,02,02,05,04,05,01,,,,,,,,50,,40.80564,-75.07627,N,,8514,2021-06-03 20:17:00,03,04,24,POINT (-75.07627 40.80564)


### Load county geometries

In [7]:
from nj_crashes.geo import get_boundary_lls, get_county_coords, get_counties, get_county_geometries
from geopandas.tools import sjoin
import geopandas as gpd
from shapely import box
import simplejson
from njdot.codes import CrashSeverity

In [8]:
%%time
cg = get_county_geometries()
cg

Loading /Users/ryan/c/neighbor-ryan/nj-crashes/tlgdb_2022_a_34_nj.gdb/


CPU times: user 159 ms, sys: 47.7 ms, total: 207 ms
Wall time: 1.04 s


,geometry
cn,
Atlantic,"MULTIPOLYGON (((-74.58640 39.30990, -74.58665 ..."
Bergen,"MULTIPOLYGON (((-73.99830 40.80848, -73.99832 ..."
Burlington,"MULTIPOLYGON (((-74.47024 39.92253, -74.47010 ..."
Camden,"MULTIPOLYGON (((-75.01333 39.95420, -75.01332 ..."
Cape May,"MULTIPOLYGON (((-74.54044 39.29945, -74.54012 ..."
Cumberland,"MULTIPOLYGON (((-75.41872 39.43250, -75.41864 ..."
Essex,"MULTIPOLYGON (((-74.13957 40.80541, -74.13959 ..."
Gloucester,"MULTIPOLYGON (((-75.11506 39.86153, -75.11508 ..."
Hudson,"MULTIPOLYGON (((-74.02064 40.73039, -74.02097 ..."


In [9]:
%%time
joined = sjoin(gdf.df[['olat', 'olon', 'geometry']], cg)
joined = joined.rename(columns={ 'index_right': 'ocn', })
joined

<timed exec>:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4269



CPU times: user 1min 21s, sys: 779 ms, total: 1min 21s
Wall time: 1min 23s


,olat,olon,geometry,ocn
id,,,,
632693,39.41477,-74.49141,POINT (-74.49141 39.41477),Atlantic
632919,39.39608,-74.49257,POINT (-74.49257 39.39608),Atlantic
633737,39.35505,-74.44556,POINT (-74.44556 39.35505),Atlantic
634226,39.35858,-74.43691,POINT (-74.43691 39.35858),Atlantic
634419,39.35287,-74.45566,POINT (-74.45566 39.35287),Atlantic
...,...,...,...,...
6077298,40.80707,-75.07559,POINT (-75.07559 40.80707),Warren
6077299,40.84466,-75.05549,POINT (-75.05549 40.84466),Warren
6077300,40.80564,-75.07627,POINT (-75.07627 40.80564),Warren


In [10]:
joined.ocn.value_counts().sort_index()

ocn
Atlantic       56825
Bergen        113569
Burlington     72611
Camden         73453
Cape May       13553
Cumberland     30709
Essex         104867
Gloucester     53142
Hudson         35605
Hunterdon      19675
Mercer         49266
Middlesex     117574
Monmouth       98022
Morris         80610
Ocean          80472
Passaic        66918
Salem          11480
Somerset       52014
Sussex         18256
Union          85890
Warren         17135
Name: count, dtype: int64

In [11]:
%%time
with_ocn = sxs(gdf.df, joined.ocn).sort_index()

CPU times: user 3.36 s, sys: 4.53 s, total: 7.89 s
Wall time: 9.95 s


In [12]:
%%time
occ = with_ocn.ocn.map(cn2cc).astype('Int64').rename('occ')
with_ocn = sxs(with_ocn, occ)
with_ocn

CPU times: user 3.55 s, sys: 6.37 s, total: 9.91 s
Wall time: 12.9 s


,year,cc,mc,case,cn,mn,pdc,pdn,station,tk,ti,pk,pi,severity,Intersection,Alcohol Involved,HazMat Involved,crash_type,tv,road,road_direction,Route,Route Suffix,sri,mp,Road System,Road Character,Road Surface Type,Surface Condition,Light Condition,Environmental Condition,Road Divided By,Temporary Traffic Control Zone,Distance To Cross Street,Unit Of Measurement,Direction From Cross Street,cross_street,Is Ramp,Ramp To/From Route Name,Ramp To/From Route Direction,Posted Speed,Posted Speed Cross Street,olat,olon,Cell Phone In Use Flag,Other Property Damage,Reporting Badge No.,dt,Road Horizontal Alignment,Road Grade,First Harmful Event,geometry,ocn,occ
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,2001,1,1,#2001-17846,Atlantic,Absecon City,01,Absecon City,MUNICIPAL COMP?,0,0,0,0,p,B,False,False,03,2,CALDERON AVENUE,,,,,NaN,07,02,02,01,06,01,05,01,100,FE,N,RT 30,,,,25,,NaN,NaN,N,NEVER SAW V-1 MINOR DAMAGE - NO INJURIES REPOR...,830,2001-12-21 18:34:00,NaN,NaN,NaN,POINT EMPTY,NaN,<NA>
1,2001,1,1,01-00029,Atlantic,Absecon City,1,Absecon,,0,0,0,0,p,B,False,False,06,2,RITZ DRIVE,,,,,NaN,07,01,02,03,06,01,05,01,,,,,,,,25,,NaN,NaN,N,,836,2001-01-01 09:30:00,NaN,NaN,NaN,POINT EMPTY,NaN,<NA>
2,2001,1,1,01-004615,Atlantic,Absecon City,1,Absecon,,0,4,0,0,i,I,False,False,03,2,MORTON AVENUE,,,,,NaN,07,01,02,01,01,01,05,01,,AT,,NEW YORK AVENUE,,,,25,,NaN,NaN,N,,836,2001-04-10 14:44:00,NaN,NaN,NaN,POINT EMPTY,NaN,<NA>
3,2001,1,1,01-004880,Atlantic,Absecon City,1,Absecon,,0,1,0,0,i,B,False,False,01,2,RT 30,,30,,00000030__,51.10,02,01,02,01,01,01,05,01,,,,,,,,45,,NaN,NaN,N,WITNESS-PETRIA GIBSON AND ANGELO HERSH,886,2001-04-15 13:56:00,NaN,NaN,NaN,POINT EMPTY,NaN,<NA>
4,2001,1,1,01-004912,Atlantic,Absecon City,1,Absecon,,0,0,0,0,p,I,False,False,01,2,CR 651,,651,,,NaN,05,02,02,02,01,02,05,01,,AT,,CR 630,,,,35,,NaN,NaN,N,*BETH VEHICLES-RELIANCE INSURANCE CO.(215)864-...,836,2001-04-16 10:29:00,NaN,NaN,NaN,POINT EMPTY,NaN,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6077298,2021,21,23,B150-2021-00117A,Warren,White Twp,02,New Jersey State Police,WASHINGTON - SQ,0,0,0,0,p,I,False,False,11,1,WARREN COUNTY 620,N,620,,21000620__,0.00,05,,02,01,06,01,05,01,,AT,,ROUTE 519 / S BRIDGEVILLE RD,,,,50,50,40.80707,-75.07559,N,Damage to 10ft of Country road guardrail.,7837,2021-05-26 21:08:00,02,04,46,POINT (-75.07559 40.80707),Warren,21
6077299,2021,21,23,B150-2021-00118A,Warren,White Twp,02,New Jersey State Police,WASHINGTON - SQ,0,0,0,0,p,I,False,False,01,2,MANUNKA CHUNK RD / UPPER SEREPTA RD **,W,,,21231034__,1.73,07,,02,01,01,01,05,01,,AT,,MASSENATTS RD,,,,40,50,40.84466,-75.05549,N,,8610,2021-05-25 07:26:00,01,04,26,POINT (-75.05549 40.84466),Warren,21
6077300,2021,21,23,B150-2021-00128A,Warren,White Twp,02,New Jersey State Police,WASHINGTON - SQ,0,0,0,0,p,B,False,False,12,1,ROUTE 519,S,519,,00000519__,44.00,05,,02,02,05,04,05,01,,,,,,,,50,,40.80564,-75.07627,N,,8514,2021-06-03 20:17:00,03,04,24,POINT (-75.07627 40.80564),Warren,21


In [13]:
assert with_ocn.index.tolist() == list(range(len(with_ocn)))

In [14]:
ill = Crashes(with_ocn).mp_lls(append=True)
ill

,year,cc,mc,case,cn,mn,pdc,pdn,station,tk,ti,pk,pi,severity,Intersection,Alcohol Involved,HazMat Involved,crash_type,tv,road,road_direction,Route,Route Suffix,sri,mp,Road System,Road Character,Road Surface Type,Surface Condition,Light Condition,Environmental Condition,Road Divided By,Temporary Traffic Control Zone,Distance To Cross Street,Unit Of Measurement,Direction From Cross Street,cross_street,Is Ramp,Ramp To/From Route Name,Ramp To/From Route Direction,Posted Speed,Posted Speed Cross Street,olat,olon,Cell Phone In Use Flag,Other Property Damage,Reporting Badge No.,dt,Road Horizontal Alignment,Road Grade,First Harmful Event,geometry,ocn,occ,reason,ilon,ilat
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,2001,1,1,#2001-17846,Atlantic,Absecon City,01,Absecon City,MUNICIPAL COMP?,0,0,0,0,p,B,False,False,03,2,CALDERON AVENUE,,,,,NaN,07,02,02,01,06,01,05,01,100,FE,N,RT 30,,,,25,,NaN,NaN,N,NEVER SAW V-1 MINOR DAMAGE - NO INJURIES REPOR...,830,2001-12-21 18:34:00,NaN,NaN,NaN,POINT EMPTY,NaN,<NA>,No MP,NaN,NaN
1,2001,1,1,01-00029,Atlantic,Absecon City,1,Absecon,,0,0,0,0,p,B,False,False,06,2,RITZ DRIVE,,,,,NaN,07,01,02,03,06,01,05,01,,,,,,,,25,,NaN,NaN,N,,836,2001-01-01 09:30:00,NaN,NaN,NaN,POINT EMPTY,NaN,<NA>,No MP,NaN,NaN
2,2001,1,1,01-004615,Atlantic,Absecon City,1,Absecon,,0,4,0,0,i,I,False,False,03,2,MORTON AVENUE,,,,,NaN,07,01,02,01,01,01,05,01,,AT,,NEW YORK AVENUE,,,,25,,NaN,NaN,N,,836,2001-04-10 14:44:00,NaN,NaN,NaN,POINT EMPTY,NaN,<NA>,No MP,NaN,NaN
3,2001,1,1,01-004880,Atlantic,Absecon City,1,Absecon,,0,1,0,0,i,B,False,False,01,2,RT 30,,30,,00000030__,51.10,02,01,02,01,01,01,05,01,,,,,,,,45,,NaN,NaN,N,WITNESS-PETRIA GIBSON AND ANGELO HERSH,886,2001-04-15 13:56:00,NaN,NaN,NaN,POINT EMPTY,NaN,<NA>,NaN,-74.512308,39.432705
4,2001,1,1,01-004912,Atlantic,Absecon City,1,Absecon,,0,0,0,0,p,I,False,False,01,2,CR 651,,651,,,NaN,05,02,02,02,01,02,05,01,,AT,,CR 630,,,,35,,NaN,NaN,N,*BETH VEHICLES-RELIANCE INSURANCE CO.(215)864-...,836,2001-04-16 10:29:00,NaN,NaN,NaN,POINT EMPTY,NaN,<NA>,No MP,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6077298,2021,21,23,B150-2021-00117A,Warren,White Twp,02,New Jersey State Police,WASHINGTON - SQ,0,0,0,0,p,I,False,False,11,1,WARREN COUNTY 620,N,620,,21000620__,0.00,05,,02,01,06,01,05,01,,AT,,ROUTE 519 / S BRIDGEVILLE RD,,,,50,50,40.80707,-75.07559,N,Damage to 10ft of Country road guardrail.,7837,2021-05-26 21:08:00,02,04,46,POINT (-75.07559 40.80707),Warren,21,NaN,-75.075607,40.807156
6077299,2021,21,23,B150-2021-00118A,Warren,White Twp,02,New Jersey State Police,WASHINGTON - SQ,0,0,0,0,p,I,False,False,01,2,MANUNKA CHUNK RD / UPPER SEREPTA RD **,W,,,21231034__,1.73,07,,02,01,01,01,05,01,,AT,,MASSENATTS RD,,,,40,50,40.84466,-75.05549,N,,8610,2021-05-25 07:26:00,01,04,26,POINT (-75.05549 40.84466),Warren,21,NaN,-75.056122,40.844345
6077300,2021,21,23,B150-2021-00128A,Warren,White Twp,02,New Jersey State Police,WASHINGTON - SQ,0,0,0,0,p,B,False,False,12,1,ROUTE 519,S,519,,00000519__,44.00,05,,02,02,05,04,05,01,,,,,,,,50,,40.80564,-75.07627,N,,8514,2021-06-03 20:17:00,03,04,24,POINT (-75.07627 40.80564),Warren,21,NaN,-75.015149,40.863108


In [15]:
%%time
igdf = Crashes(ill).gdf('i')
igdf

CPU times: user 5.84 s, sys: 12.2 s, total: 18 s
Wall time: 32.4 s


,year,cc,mc,case,cn,mn,pdc,pdn,station,tk,ti,pk,pi,severity,Intersection,Alcohol Involved,HazMat Involved,crash_type,tv,road,road_direction,Route,Route Suffix,sri,mp,Road System,Road Character,Road Surface Type,Surface Condition,Light Condition,Environmental Condition,Road Divided By,Temporary Traffic Control Zone,Distance To Cross Street,Unit Of Measurement,Direction From Cross Street,cross_street,Is Ramp,Ramp To/From Route Name,Ramp To/From Route Direction,Posted Speed,Posted Speed Cross Street,olat,olon,Cell Phone In Use Flag,Other Property Damage,Reporting Badge No.,dt,Road Horizontal Alignment,Road Grade,First Harmful Event,geometry,ocn,occ,reason,ilon,ilat
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,2001,1,1,#2001-17846,Atlantic,Absecon City,01,Absecon City,MUNICIPAL COMP?,0,0,0,0,p,B,False,False,03,2,CALDERON AVENUE,,,,,NaN,07,02,02,01,06,01,05,01,100,FE,N,RT 30,,,,25,,NaN,NaN,N,NEVER SAW V-1 MINOR DAMAGE - NO INJURIES REPOR...,830,2001-12-21 18:34:00,NaN,NaN,NaN,POINT EMPTY,NaN,<NA>,No MP,NaN,NaN
1,2001,1,1,01-00029,Atlantic,Absecon City,1,Absecon,,0,0,0,0,p,B,False,False,06,2,RITZ DRIVE,,,,,NaN,07,01,02,03,06,01,05,01,,,,,,,,25,,NaN,NaN,N,,836,2001-01-01 09:30:00,NaN,NaN,NaN,POINT EMPTY,NaN,<NA>,No MP,NaN,NaN
2,2001,1,1,01-004615,Atlantic,Absecon City,1,Absecon,,0,4,0,0,i,I,False,False,03,2,MORTON AVENUE,,,,,NaN,07,01,02,01,01,01,05,01,,AT,,NEW YORK AVENUE,,,,25,,NaN,NaN,N,,836,2001-04-10 14:44:00,NaN,NaN,NaN,POINT EMPTY,NaN,<NA>,No MP,NaN,NaN
3,2001,1,1,01-004880,Atlantic,Absecon City,1,Absecon,,0,1,0,0,i,B,False,False,01,2,RT 30,,30,,00000030__,51.10,02,01,02,01,01,01,05,01,,,,,,,,45,,NaN,NaN,N,WITNESS-PETRIA GIBSON AND ANGELO HERSH,886,2001-04-15 13:56:00,NaN,NaN,NaN,POINT (-74.51231 39.43270),NaN,<NA>,NaN,-74.512308,39.432705
4,2001,1,1,01-004912,Atlantic,Absecon City,1,Absecon,,0,0,0,0,p,I,False,False,01,2,CR 651,,651,,,NaN,05,02,02,02,01,02,05,01,,AT,,CR 630,,,,35,,NaN,NaN,N,*BETH VEHICLES-RELIANCE INSURANCE CO.(215)864-...,836,2001-04-16 10:29:00,NaN,NaN,NaN,POINT EMPTY,NaN,<NA>,No MP,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6077298,2021,21,23,B150-2021-00117A,Warren,White Twp,02,New Jersey State Police,WASHINGTON - SQ,0,0,0,0,p,I,False,False,11,1,WARREN COUNTY 620,N,620,,21000620__,0.00,05,,02,01,06,01,05,01,,AT,,ROUTE 519 / S BRIDGEVILLE RD,,,,50,50,40.80707,-75.07559,N,Damage to 10ft of Country road guardrail.,7837,2021-05-26 21:08:00,02,04,46,POINT (-75.07561 40.80716),Warren,21,NaN,-75.075607,40.807156
6077299,2021,21,23,B150-2021-00118A,Warren,White Twp,02,New Jersey State Police,WASHINGTON - SQ,0,0,0,0,p,I,False,False,01,2,MANUNKA CHUNK RD / UPPER SEREPTA RD **,W,,,21231034__,1.73,07,,02,01,01,01,05,01,,AT,,MASSENATTS RD,,,,40,50,40.84466,-75.05549,N,,8610,2021-05-25 07:26:00,01,04,26,POINT (-75.05612 40.84434),Warren,21,NaN,-75.056122,40.844345
6077300,2021,21,23,B150-2021-00128A,Warren,White Twp,02,New Jersey State Police,WASHINGTON - SQ,0,0,0,0,p,B,False,False,12,1,ROUTE 519,S,519,,00000519__,44.00,05,,02,02,05,04,05,01,,,,,,,,50,,40.80564,-75.07627,N,,8514,2021-06-03 20:17:00,03,04,24,POINT (-75.01515 40.86311),Warren,21,NaN,-75.015149,40.863108


In [16]:
type(igdf)

njdot.crashes.Crashes

In [17]:
%%time
ij = sjoin(igdf.df[['geometry']], cg)
ij = ij.rename(columns={ 'index_right': 'icn', })
ij

<timed exec>:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4269



CPU times: user 3min 39s, sys: 1.56 s, total: 3min 40s
Wall time: 3min 47s


,geometry,icn
id,,
3,POINT (-74.51231 39.43270),Atlantic
5,POINT (-74.49717 39.42692),Atlantic
6,POINT (-74.50114 39.42445),Atlantic
8,POINT (-74.51094 39.43397),Atlantic
9,POINT (-74.51274 39.43301),Atlantic
...,...,...
6056506,POINT (-74.62165 41.18961),Sussex
6056507,POINT (-74.62955 41.22200),Sussex
6074721,POINT (-74.77471 40.91620),Sussex


In [18]:
%%time
with_cns = sxs(igdf.df, ij.icn).sort_index()
icc = with_cns.icn.map(cn2cc).astype('Int64').rename('icc')
with_cns = sxs(with_cns, icc)
with_cns

CPU times: user 6 s, sys: 10.3 s, total: 16.3 s
Wall time: 26.6 s


,year,cc,mc,case,cn,mn,pdc,pdn,station,tk,ti,pk,pi,severity,Intersection,Alcohol Involved,HazMat Involved,crash_type,tv,road,road_direction,Route,Route Suffix,sri,mp,Road System,Road Character,Road Surface Type,Surface Condition,Light Condition,Environmental Condition,Road Divided By,Temporary Traffic Control Zone,Distance To Cross Street,Unit Of Measurement,Direction From Cross Street,cross_street,Is Ramp,Ramp To/From Route Name,Ramp To/From Route Direction,Posted Speed,Posted Speed Cross Street,olat,olon,Cell Phone In Use Flag,Other Property Damage,Reporting Badge No.,dt,Road Horizontal Alignment,Road Grade,First Harmful Event,geometry,ocn,occ,reason,ilon,ilat,icn,icc
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,2001,1,1,#2001-17846,Atlantic,Absecon City,01,Absecon City,MUNICIPAL COMP?,0,0,0,0,p,B,False,False,03,2,CALDERON AVENUE,,,,,NaN,07,02,02,01,06,01,05,01,100,FE,N,RT 30,,,,25,,NaN,NaN,N,NEVER SAW V-1 MINOR DAMAGE - NO INJURIES REPOR...,830,2001-12-21 18:34:00,NaN,NaN,NaN,POINT EMPTY,NaN,<NA>,No MP,NaN,NaN,NaN,<NA>
1,2001,1,1,01-00029,Atlantic,Absecon City,1,Absecon,,0,0,0,0,p,B,False,False,06,2,RITZ DRIVE,,,,,NaN,07,01,02,03,06,01,05,01,,,,,,,,25,,NaN,NaN,N,,836,2001-01-01 09:30:00,NaN,NaN,NaN,POINT EMPTY,NaN,<NA>,No MP,NaN,NaN,NaN,<NA>
2,2001,1,1,01-004615,Atlantic,Absecon City,1,Absecon,,0,4,0,0,i,I,False,False,03,2,MORTON AVENUE,,,,,NaN,07,01,02,01,01,01,05,01,,AT,,NEW YORK AVENUE,,,,25,,NaN,NaN,N,,836,2001-04-10 14:44:00,NaN,NaN,NaN,POINT EMPTY,NaN,<NA>,No MP,NaN,NaN,NaN,<NA>
3,2001,1,1,01-004880,Atlantic,Absecon City,1,Absecon,,0,1,0,0,i,B,False,False,01,2,RT 30,,30,,00000030__,51.10,02,01,02,01,01,01,05,01,,,,,,,,45,,NaN,NaN,N,WITNESS-PETRIA GIBSON AND ANGELO HERSH,886,2001-04-15 13:56:00,NaN,NaN,NaN,POINT (-74.51231 39.43270),NaN,<NA>,NaN,-74.512308,39.432705,Atlantic,1
4,2001,1,1,01-004912,Atlantic,Absecon City,1,Absecon,,0,0,0,0,p,I,False,False,01,2,CR 651,,651,,,NaN,05,02,02,02,01,02,05,01,,AT,,CR 630,,,,35,,NaN,NaN,N,*BETH VEHICLES-RELIANCE INSURANCE CO.(215)864-...,836,2001-04-16 10:29:00,NaN,NaN,NaN,POINT EMPTY,NaN,<NA>,No MP,NaN,NaN,NaN,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6077298,2021,21,23,B150-2021-00117A,Warren,White Twp,02,New Jersey State Police,WASHINGTON - SQ,0,0,0,0,p,I,False,False,11,1,WARREN COUNTY 620,N,620,,21000620__,0.00,05,,02,01,06,01,05,01,,AT,,ROUTE 519 / S BRIDGEVILLE RD,,,,50,50,40.80707,-75.07559,N,Damage to 10ft of Country road guardrail.,7837,2021-05-26 21:08:00,02,04,46,POINT (-75.07561 40.80716),Warren,21,NaN,-75.075607,40.807156,Warren,21
6077299,2021,21,23,B150-2021-00118A,Warren,White Twp,02,New Jersey State Police,WASHINGTON - SQ,0,0,0,0,p,I,False,False,01,2,MANUNKA CHUNK RD / UPPER SEREPTA RD **,W,,,21231034__,1.73,07,,02,01,01,01,05,01,,AT,,MASSENATTS RD,,,,40,50,40.84466,-75.05549,N,,8610,2021-05-25 07:26:00,01,04,26,POINT (-75.05612 40.84434),Warren,21,NaN,-75.056122,40.844345,Warren,21
6077300,2021,21,23,B150-2021-00128A,Warren,White Twp,02,New Jersey State Police,WASHINGTON - SQ,0,0,0,0,p,B,False,False,12,1,ROUTE 519,S,519,,00000519__,44.00,05,,02,02,05,04,05,01,,,,,,,,50,,40.80564,-75.07627,N,,8514,2021-06-03 20:17:00,03,04,24,POINT (-75.01515 40.86311),Warren,21,NaN,-75.015149,40.863108,Warren,21


In [19]:
with_cns.groupby(['cc', 'mc'])['mn'].apply(lambda s: s.value_counts())

cc  mc                   
1   1   Absecon City          7462
    2   Atlantic City        43797
    3   Brigantine City       2958
    4   Buena Boro            2057
    5   Buena Vista Twp       4150
                             ...  
21  19  Phillipsburg Town    10066
    20  Pohatcong Twp         5204
    21  Washington Boro       2757
    22  Washington Twp        3853
    23  White Twp             2918
Name: mn, Length: 571, dtype: int64

In [20]:
assert with_cns.index.tolist() == list(range(len(with_cns)))

In [21]:
type(with_cns)

geopandas.geodataframe.GeoDataFrame

In [24]:
ill_cols = ['icc', 'ilat', 'ilon']
oll_cols = ['occ', 'olat', 'olon']
sql_cols = [
    'dt',
    *pk_cols,
    *ksi_renames.values(),
    'crash_type', 'severity',
    *road_cols,
    *ill_cols,
    *oll_cols,
]

In [25]:
sd = with_cns[sql_cols]
sd

,dt,year,cc,mc,case,tk,ti,pk,pi,tv,crash_type,severity,sri,mp,road,road_direction,cross_street,icc,ilat,ilon,occ,olat,olon
id,,,,,,,,,,,,,,,,,,,,,,,
0,2001-12-21 18:34:00,2001,1,1,#2001-17846,0,0,0,0,2,03,p,,NaN,CALDERON AVENUE,,RT 30,<NA>,NaN,NaN,<NA>,NaN,NaN
1,2001-01-01 09:30:00,2001,1,1,01-00029,0,0,0,0,2,06,p,,NaN,RITZ DRIVE,,,<NA>,NaN,NaN,<NA>,NaN,NaN
2,2001-04-10 14:44:00,2001,1,1,01-004615,0,4,0,0,2,03,i,,NaN,MORTON AVENUE,,NEW YORK AVENUE,<NA>,NaN,NaN,<NA>,NaN,NaN
3,2001-04-15 13:56:00,2001,1,1,01-004880,0,1,0,0,2,01,i,00000030__,51.10,RT 30,,,1,39.432705,-74.512308,<NA>,NaN,NaN
4,2001-04-16 10:29:00,2001,1,1,01-004912,0,0,0,0,2,01,p,,NaN,CR 651,,CR 630,<NA>,NaN,NaN,<NA>,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6077298,2021-05-26 21:08:00,2021,21,23,B150-2021-00117A,0,0,0,0,1,11,p,21000620__,0.00,WARREN COUNTY 620,N,ROUTE 519 / S BRIDGEVILLE RD,21,40.807156,-75.075607,21,40.80707,-75.07559
6077299,2021-05-25 07:26:00,2021,21,23,B150-2021-00118A,0,0,0,0,2,01,p,21231034__,1.73,MANUNKA CHUNK RD / UPPER SEREPTA RD **,W,MASSENATTS RD,21,40.844345,-75.056122,21,40.84466,-75.05549
6077300,2021-06-03 20:17:00,2021,21,23,B150-2021-00128A,0,0,0,0,1,12,p,00000519__,44.00,ROUTE 519,S,,21,40.863108,-75.015149,21,40.80564,-75.07627


In [26]:
from nj_crashes.utils import sql

def write_sql(df, db_path, rm=True, tbl='crashes'):
    return sql.write(
        df, tbl=tbl, db_path=db_path,
        idxs=[
            ('severity', 'dt', 'cc', 'mc'),
            ('severity', 'ilat', 'ilon'),
            ('severity', 'icc', 'dt'),
        ]
    )

In [27]:
%%time
cur = write_sql(sd, CRASHES_DB)
cur

Writing 6077303 rows to /Users/ryan/c/neighbor-ryan/nj-crashes/www/public/njdot/crashes.db


CPU times: user 1min 22s, sys: 27.4 s, total: 1min 49s
Wall time: 2min 21s


In [28]:
cur.execute("Select * from SQLite_master")
idxs = cur.fetchall()
idxs

[('table',
  'crashes',
  'crashes',
  2,
  'CREATE TABLE crashes (\n\tid BIGINT, \n\tdt DATETIME, \n\tyear INTEGER, \n\tcc BIGINT, \n\tmc BIGINT, \n\t"case" TEXT, \n\ttk BIGINT, \n\tti BIGINT, \n\tpk BIGINT, \n\tpi BIGINT, \n\ttv BIGINT, \n\tcrash_type TEXT, \n\tseverity TEXT, \n\tsri TEXT, \n\tmp FLOAT, \n\troad TEXT, \n\troad_direction TEXT, \n\tcross_street TEXT, \n\ticc BIGINT, \n\tilat FLOAT, \n\tilon FLOAT, \n\tocc BIGINT, \n\tolat FLOAT, \n\tolon FLOAT\n)'),
 ('index',
  'ix_crashes_id',
  'crashes',
  3,
  'CREATE INDEX ix_crashes_id ON crashes (id)'),
 ('index',
  'severity_dt_cc_mc',
  'crashes',
  209167,
  'CREATE INDEX severity_dt_cc_mc ON crashes(severity, dt, cc, mc)'),
 ('index',
  'severity_ilat_ilon',
  'crashes',
  270579,
  'CREATE INDEX severity_ilat_ilon ON crashes(severity, ilat, ilon)'),
 ('index',
  'severity_icc_dt',
  'crashes',
  302082,
  'CREATE INDEX severity_icc_dt ON crashes(severity, icc, dt)')]

In [29]:
%%time
with_cns.drop(columns='geometry').to_parquet(CNS)

CPU times: user 12.4 s, sys: 4.7 s, total: 17.1 s
Wall time: 22.9 s
